# The wekeo provider in EODAG

This tutorial will show you how use EODAG to download data from [WEkEO](https://www.wekeo.eu/) using `eodag`. WEkEO offers a wide range of Copernicus and Sentinel products decomposed into three providers classified according to their kind of data:

* `wekeo_main`: Sentinel and some Copernicus data
* `wekeo_ecmwf`: Copernicus data from ECMWF
* `wekeo_cmems`: Copernicus Marine data

## Registration

For both, searching and downloading data, authentication is required. Therefore, you first have to create an account on [https://www.wekeo.eu/](https://www.wekeo.eu/) to get a `username` and a `password` which have to be added to the EODAG configuration (see [Configure EODAG](../../getting_started_guide/configure.rst)). After registration you also have to accept the Terms and Conditions for the datasets you want to use to be able to retrieve data from EODAG. This has to be done once per user and licence by executing the following code where licence_name has to be set to the desired licence (see list below):

In [1]:
import requests
from eodag import EODataAccessGateway

# instantiate EODAG API
dag = EODataAccessGateway()

# Authenticate on the wekeo adapted to your request, here wekeo_main
search_plugin = next(dag._plugins_manager.get_search_plugins(provider="wekeo_main"))
auth = dag._plugins_manager.get_auth_plugin(search_plugin).authenticate()
licence_name = "EUMETSAT_Copernicus_Data_Licence"

# Use your authentication to accept Terms and Conditions
response = requests.put(
    f"https://gateway.prod.wekeo2.eu/hda-broker/api/v1/termsaccepted/{licence_name}",
    auth=auth,
)

Depending on which data you want to use, different licences have to be accepted. If you want to use all the collections available in EODAG for the provider `wekeo_main`, the following licences have to be accepted:
      
* EUMETSAT_Copernicus_Data_Licence
* Copernicus_Land_Monitoring_Service_Data_Policy
* Copernicus_Sentinel_License
* Copernicus_DEM_Instance_COP-DEM-GLO-30-F_Global_30m
* Copernicus_DEM_Instance_COP-DEM-GLO-90-F_Global_90m

To access the products of the provider `wekeo_ecmwf` or `wekeo_cmems` the licence Copernicus_ECMWF_License or Copernicus_Marine_Service_Product_License have to be accepted respectively.

## Search and download data

Many collections are available for the three `wekeo` providers, here let's see for `wekeo_ecmwf`:

In [2]:
wekeo_collections = dag.list_collections("wekeo_ecmwf")
len(wekeo_collections)

37

In [ ]:
[pt.id for pt in wekeo_collections]

['AG_ERA5',
 'CAMS_EAC4',
 'CAMS_EAC4_MONTHLY',
 'CAMS_EU_AIR_QUALITY_FORECAST',
 'CAMS_EU_AIR_QUALITY_RE',
 'CAMS_GAC_FORECAST',
 'CAMS_GFE_GFAS',
 'CAMS_GLOBAL_EMISSIONS',
 'CAMS_GREENHOUSE_EGG4',
 'CAMS_GREENHOUSE_EGG4_MONTHLY',
 'CAMS_GREENHOUSE_INVERSION',
 'CAMS_GRF',
 'CAMS_GRF_AUX',
 'CAMS_SOLAR_RADIATION',
 'ERA5_LAND',
 'ERA5_LAND_MONTHLY',
 'ERA5_PL',
 'ERA5_PL_MONTHLY',
 'ERA5_SL',
 'ERA5_SL_MONTHLY',
 'FIRE_HISTORICAL',
 'GLACIERS_DIST_RANDOLPH',
 'GRIDDED_GLACIERS_MASS_CHANGE',
 'SATELLITE_CARBON_DIOXIDE',
 'SATELLITE_FIRE_BURNED_AREA',
 'SATELLITE_METHANE',
 'SATELLITE_SEA_ICE_CONCENTRATION',
 'SATELLITE_SEA_ICE_EDGE_TYPE',
 'SATELLITE_SEA_ICE_THICKNESS',
 'SATELLITE_SEA_LEVEL_GLOBAL',
 'SEASONAL_MONTHLY_PL',
 'SEASONAL_MONTHLY_SL',
 'SEASONAL_ORIGINAL_PL',
 'SEASONAL_ORIGINAL_SL',
 'SEASONAL_POSTPROCESSED_PL',
 'SEASONAL_POSTPROCESSED_SL',
 'UERRA_EUROPE_SL']

Check queryables for data on one of these collections:

In [4]:
dag.list_queryables(
    collection="CAMS_GFE_GFAS", 
    provider="wekeo_ecmwf"
)

QueryablesDict (4) - additional_properties=False
"str,"
"FieldInfo(annotation=NoneType, required=False, default='2003-01-01T00:00:00Z', alias='end_datetime', alias_priority=2, description=""Date/time as string in ISO 8601 format (e.g. '2024-06-10T12:00:00Z')"")"
"str,"
"FieldInfo(annotation=NoneType, required=False, default='2003-01-01T00:00:00Z', alias='start_datetime', alias_priority=2, description=""Date/time as string in ISO 8601 format (e.g. '2024-06-10T12:00:00Z')"")"
"typing.Literal['grib', 'netcdf'],"
"FieldInfo(annotation=NoneType, required=True, alias='ecmwf:data_format', alias_priority=2, title='data_format')"
"typing.Literal['altitude_of_plume_bottom', 'altitude_of_plume_top', 'injection_height', 'mean_altitude_of_maximum_injection', 'wildfire_combustion_rate', 'wildfire_flux_of_acetaldehyde', 'wildfire_flux_of_acetone', 'wildfire_flux_of_ammonia', 'wildfire_flux_of_benzene', 'wildfire_flux_of_black_carbon', 'wildfire_flux_of_butanes', 'wildfire_flux_of_butenes', 'wildfire_flux_of_carbon_dioxide', 'wildfire_flux_of_carbon_monoxide', 'wildfire_flux_of_dimethyl_sulfide', 'wildfire_flux_of_ethane', 'wildfire_flux_of_ethanol', 'wildfire_flux_of_ethene', 'wildfire_flux_of_formaldehyde', 'wildfire_flux_of_heptane', 'wildfire_flux_of_hexanes', 'wildfire_flux_of_hexene', 'wildfire_flux_of_higher_alkanes', 'wildfire_flux_of_higher_alkenes', 'wildfire_flux_of_hydrogen', 'wildfire_flux_of_isoprene', 'wildfire_flux_of_methane', 'wildfire_flux_of_methanol', 'wildfire_flux_of_nitrogen_oxides', 'wildfire_flux_of_nitrous_oxide', 'wildfire_flux_of_non_methane_hydrocarbons', 'wildfire_flux_of_octene', 'wildfire_flux_of_organic_carbon', 'wildfire_flux_of_particulate_matter_d_2_5_µm', 'wildfire_flux_of_pentanes', 'wildfire_flux_of_pentenes', 'wildfire_flux_of_propane', 'wildfire_flux_of_propene', 'wildfire_flux_of_sulphur_dioxide', 'wildfire_flux_of_terpenes', 'wildfire_flux_of_toluene', 'wildfire_flux_of_toluene_lump', 'wildfire_flux_of_total_carbon_in_aerosols', 'wildfire_flux_of_total_particulate_matter', 'wildfire_flux_of_xylene', 'wildfire_fraction_of_area_observed', 'wildfire_overall_flux_of_burnt_carbon', 'wildfire_radiative_power'],"
"FieldInfo(annotation=NoneType, required=True, alias='ecmwf:variable', alias_priority=2, title='variable')"


Search for data on this collection:

In [14]:
products = dag.search(
    collection="CAMS_GFE_GFAS", 
    start="2018-01-01", 
    end="2018-01-02", 
    geom=[0.25, 43.2, 2.8, 43.9],
    provider="wekeo_ecmwf",
    **{
        'ecmwf:variable': 'wildfire_flux_of_total_carbon_in_aerosols',
        'ecmwf:data_format': 'grib',
    }
)
products

SearchResult([EOProduct(id=CAMS_GFE_GFAS_ORDERABLE_e6bed340557b2d9d6f914c7810c14cde, provider=wekeo_ecmwf)])

Download found product to `/tmp`, changing wait time to 12s (0.2') between product order and download retries:

In [15]:
path = products[0].download(output_dir="/tmp", wait=0.2)
path

0.00B [00:00, ?B/s]

'/tmp/CAMS_GFE_GFAS_68f62e55b6f16ef4c6509884'

## The WEkEO data viewer

If you go to the WEkEO [homepage](https://www.wekeo.eu) and click on "Data" in the top menu, you can search for products online. You can choose a collection (called dataset in WEkEO) by clicking on the "+" button next to "Layers" on the left side. Then you can choose some filter parameters and create a data request job which will show up in the next tab where the products can be downloaded. The data viewer currently still uses the old version of the API where data requests have to be created to search product. In contrast, the latest version of EODAG already uses the new API (https://gateway.prod.wekeo2.eu/hda-broker/api/v1) where a direct search without data requests is possible.